In [133]:
'''利用采样函数，设定转换周期period_type  转换为周是'W',月'M',季度线'Q',五分钟'5min',12天12D'''
def get_period_date(peroid,start_date, end_date):
    stock_data = get_price('000001.XSHE',start_date,end_date,'daily',fields=['close'])
    #types = stock_data.index.dtype 查看index的数据类型
    stock_data['date']=stock_data.index #resample按照index自然日采样，获取日期临时值
    start_date_0 = stock_data['date'][0].strftime("%Y-%m-%d")
    period_stock_data=stock_data.resample(peroid,how= 'last').dropna() #进行转换，周线的每个变量都等于那一周中最后一个交易日的变量值
    date_list = period_stock_data['date'].apply(lambda s: s.strftime('%Y-%m-%d')).values.tolist() #numpy.arrange 转 list
    if start_date_0 not in date_list:        
        date_list.insert(0, start_date_0)
    return date_list
def generate_factors(stock_list, start_date, end_date):
    date_list = get_period_date('1D', start_date, end_date)
    q = query(valuation.circulating_market_cap,valuation.code).filter(valuation.code.in_(stock_list))   
    fundamental_fator = get_fundamentals(q, date_list[-1])
    fundamental_fator.index = fundamental_fator['code']
    cir_cap = fundamental_fator['circulating_market_cap']
    factor_df = pd.DataFrame(columns = ['mf','ic','mfp'])
    for stock in stock_list:
        try:
            df = get_price(stock,start_date = start_date, end_date = end_date, frequency='1m', fields=['close','volume','money'])
        except:
            df = get_price(stock,start_date = start_date, end_date = end_date,frequency='1m', fields=['close','volume','money'])
        df_delta = df['close'].shift(1)
        df["mf"] = df['volume'] * df['close'] * abs(df['close'] - df_delta)/(df['close'] - df_delta)
        
        factor_df.loc[stock,'mf'] = df['mf'].sum()/10000 #mf:资金流净额，单位：万元
        
        factor_df.loc[stock,'ic'] =abs(df['mf'].sum()/10000/df['money'].sum() * 10000) #ic:资金流信息含量，mf/交易额
        
        factor_df.loc[stock,'mfp'] = abs(cir_cap[stock]*100000000/df['mf'].sum()) #mfp:资金流杠杆倍数，流通市值/mf
    factor_df = factor_df.dropna( how= 'any')
    return factor_df

def score_stock(factor_df):
    effective_factors = {'mf':False,'ic':True,'mfp':False}
    factor_df = factor_df[factor_df['ic']>0.10]
    score = {}
    for factor, value in effective_factors.items():
        score[factor] = factor_df[factor].rank(ascending = value)
    score_df = pd.DataFrame(score)
    score_stock = list(pd.DataFrame(score).T.sum().sort_values(ascending = False).index)
    return score_stock

start_date = '2020-02-03'
end_date = '2020-03-03'
group_quantile = 0.2
stock_list = get_index_stocks('000300.XSHG', start_date)
factor_df = generate_factors(stock_list, start_date, end_date)
result_stock = score_stock(factor_df)
group_count = int(group_quantile * len(result_stock))
group_list = []
for i in range(int(1.0/group_quantile)):
    start = i * group_count
    end = (i + 1) * group_count
    group_list.append(stock_list[start:end])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  import sys


In [137]:
def generate_factors1(stock_list, start_date, end_date):
    date_list = get_period_date('1D', start_date, end_date)
    q = query(valuation.circulating_market_cap,valuation.code).filter(valuation.code.in_(stock_list))   
    fundamental_fator = get_fundamentals(q, date_list[-1])
    fundamental_fator.index = fundamental_fator['code']
    cir_cap = fundamental_fator['circulating_market_cap']
    factor_df = pd.DataFrame(columns = ['mf','ic','mfp'])
    for stock in stock_list:
        try:
            df = get_price(stock,start_date = start_date, end_date = end_date, frequency='1m', fields=['close','volume','money'])
        except:
            df = get_price(stock,start_date = start_date, end_date = end_date,frequency='1m', fields=['close','volume','money'])
        df_delta = df['close'].shift(1)
        df["mf"] = df['volume'] * df['close'] * abs(df['close'] - df_delta)/(df['close'] - df_delta)
        
        #factor_df.loc[stock,'mf'] = df['mf'] #mf:资金流净额，单位：万元
        
        factor_df.loc[stock,'ic'] =abs(df['mf']/df['money']) #ic:资金流信息含量，mf/交易额
        
        factor_df.loc[stock,'mfp'] = abs(cir_cap[stock]/df['mf']) #mfp:资金流杠杆倍数，流通市值/mf
    factor_df = factor_df.dropna( how= 'any')
    return factor_df


In [138]:
start_date = '2020-06-09'
end_date = '2020-06-16'
group_quantile = 0.2
stock_list = ['000725.XSHE']
factor_df = generate_factors1(stock_list, start_date, end_date)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  import sys


ValueError: Incompatible indexer with Series

In [ ]:
def get_period_date(peroid,start_date, end_date):
    stock_data = get_price('000001.XSHE',start_date,end_date,'daily',fields=['close'])
    #types = stock_data.index.dtype 查看index的数据类型
    stock_data['date']=stock_data.index #resample按照index自然日采样，获取日期临时值
    start_date_0 = stock_data['date'][0].strftime("%Y-%m-%d")
    period_stock_data=stock_data.resample(peroid,how= 'last').dropna() #进行转换，周线的每个变量都等于那一周中最后一个交易日的变量值
    date_list = period_stock_data['date'].apply(lambda s: s.strftime('%Y-%m-%d')).values.tolist() #numpy.arrange 转 list
    if start_date_0 not in date_list:        
        date_list.insert(0, start_date_0)
    return date_list

In [144]:
start_date = '2020-06-09'
end_date = '2020-06-16'
group_quantile = 0.2
stock = '000725.XSHE'
factor_df = pd.DataFrame()
date_list = get_period_date('1D', start_date, end_date)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  import sys


In [141]:
df = get_price(stock,start_date = start_date, end_date = end_date, frequency='1m', fields=['close','volume','money'])
df_delta = df['close'].shift(1)
df["mf"] = df['volume'] * df['close'] * abs(df['close'] - df_delta)/(df['close'] - df_delta)

In [157]:
for i in range(len(date_list)):
    print(df['mf'].iloc[i *240:(i +1) *240].sum())
    factor_df.loc[date_list[i], 'mf'] = df['mf'].iloc[i *240:(i +1) *240].sum()

-932199693.0
1204139223.0
-862990163.0
-578067268.0
-716823930.0
0.0


In [158]:
factor_df['mf']

2020-06-09   -9.321997e+08
2020-06-10    1.204139e+09
2020-06-11   -8.629902e+08
2020-06-12   -5.780673e+08
2020-06-15   -7.168239e+08
2020-06-16    0.000000e+00
Name: mf, dtype: float64

In [159]:
df

,close,volume,money,mf
2020-06-09 09:31:00,4.64,76786500.0,356327883.0,NaN
2020-06-09 09:32:00,4.65,34097400.0,158834585.0,158552910.0
2020-06-09 09:33:00,4.64,28612600.0,133031915.0,-132762464.0
2020-06-09 09:34:00,4.61,53293300.0,246303517.0,-245682113.0
2020-06-09 09:35:00,4.63,19249200.0,88867514.0,89123796.0
2020-06-09 09:36:00,4.64,12896100.0,59809047.0,59837904.0
2020-06-09 09:37:00,4.63,22775100.0,105773120.0,-105448713.0
2020-06-09 09:38:00,4.64,13195200.0,61140464.0,61225728.0
2020-06-09 09:39:00,4.63,15102100.0,69996612.0,-69922723.0
2020-06-09 09:40:00,4.63,8954300.0,41416627.0,NaN
